In [9]:
# jupyter安装soundfile 新开terminal输入
'''
sudo yum update -y
# sudo wget http://www.mega-nerd.com/libsndfile/files/libsndfile-1.0.28.tar.gz
# tar -xzf libsndfile-1.0.28.tar.gz
cd /home/ec2-user/SageMaker/asteroid_byoc/
cd libsndfile-1.0.28
./configure --prefix=/usr --disable-static --docdir=/usr/share/doc/libsndfile-1.0.28 && make
sudo make install
'''

# 构建镜像，确认权限
'''
chmod a+x build_and_push.sh
./build_and_push.sh asteroid
/bin/bash ./helper/setup.sh
'''

In [152]:
!cat ./code/Dockerfile

FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-cpu-py36-ubuntu16.04

RUN apt-get update 
RUN apt-get -y update && apt-get install -y --no-install-recommends \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*
    
RUN wget http://www.mega-nerd.com/libsndfile/files/libsndfile-1.0.28.tar.gz
RUN tar -xzf libsndfile-1.0.28.tar.gz
RUN cd libsndfile-1.0.28 && ./configure --prefix=/usr --disable-static --docdir=/usr/share/doc/libsndfile-1.0.28 && make && make install
RUN pip install pysoundfile numpy numba Cython asteroid sagemaker-inference PyYAML pytorch-lightning==1.4.9 flask gunicorn

ENV PATH="/opt/ml/code:${PATH}"

# /opt/ml and all subdirectories are utilized by SageMaker, we use the /code subdirectory to store our user code.
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

COPY ./ /opt/ml/code
WORKDIR /opt/ml/code


In [2]:
#jupyter instance安装依赖项
!pip install librosa
!pip install asteroid 
!pip install pysoundfile

import soundfile
import librosa
import asteroid

     |████████████████████████████████| 211 kB 14.3 MB/s            
     |████████████████████████████████| 323 kB 53.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 56 kB 4.3 MB/s             
     |████████████████████████████████| 377 kB 45.5 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23308 sha256=1305dc71783842e5792d746a7574885b0f9f5e4bab59cae3786be05815fd0627
  Stored in directory: /home/ec2-user/.cache/pip/wheels/de/14/0a/863e4ed680b3204444cf486733e609d7ff7abe8fceafab67dc
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=321179 sha256=25fe9703e1fe6a5bd04f21f6a290f7159e72ad88231e0048ccd0905d3465a0d7
  Stored in directory: /home/ec2-user/.cache/pip/wheels/cf/d4/04/49d8824a42bd9f9b11d502727965b9997f0d41d2b22ae4f645
Successfully built audioread resampy
     |████████████████████████████████| 246 kB 20.0 MB/s         

In [13]:
import os
import subprocess
import sagemaker
import torch
from sagemaker import get_execution_role
from helper.utils import *
import argparse
import librosa
import torch
from scipy.signal import lfilter
import numpy as np
import scipy.io.wavfile
import soundfile as sf
import fnmatch, os, warnings
import json
from code.pred_utils import *

sagemaker_session = sagemaker.Session()
_role = get_execution_role()

sess = sagemaker.Session()
sample_rate=16000 #采样率


In [16]:
# train
from sagemaker.estimator import Estimator
import subprocess
from sagemaker import get_execution_role


role = get_execution_role()

instance_type = 'ml.p3.2xlarge'
  
print("Instance type = " + instance_type)

estimator = Estimator(role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      base_job_name = 'asteroid',
                      volume_size=50,
                      image_uri='310850127430.dkr.ecr.us-east-1.amazonaws.com/asteroid:latest')# need to change image uri

estimator.fit('s3://dns4/cv') # need to change the data path



us-east-1
310850127430.dkr.ecr.us-east-1.amazonaws.com/asteroid:latest
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  226.7MB
Step 1/9 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-cpu-py36-ubuntu16.04
 ---> e77085e0f844
Step 2/9 : RUN apt-get update
 ---> Using cache
 ---> 5c2fa9faa2f6
Step 3/9 : RUN wget http://www.mega-nerd.com/libsndfile/file

KeyboardInterrupt: 

In [43]:
# deploy
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor
import json
import time

role = get_execution_role()

model_path='s3://sagemaker-us-east-1-310850127430/asteroid-2022-02-04-13-58-24-521/output/model.tar.gz' # 需要修改模型路径
model_name='asteroid'+ time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

model = Model(
    image_uri="310850127430.dkr.ecr.us-east-1.amazonaws.com/asteroid:latest",
    name=model_name,
    role=role,
    model_data=model_path
)

endpoint_name = f'endpoint-{model_name}-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
predictor = model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)


-

KeyboardInterrupt: 

Gracefully stopping... (press Ctrl+C again to force)


In [70]:
#调用endpoint

import boto3
import json
import librosa
from tqdm import tqdm
from helper.utils import *
from code.pred_utils import *
import boto3

endpoint_name = 'endpoint-asteroid2022-02-15-14-54-45-2022-02-15-14-54-45' #需要修改endpoint_name

client = boto3.client('sagemaker-runtime')

# 数据处理function
def preprocess_wav(filename, fs = 16000):
    x, fs = librosa.load(filename, sr =fs)
    return x, fs


def pad_list(xs, pad_value):
    n_batch = len(xs)
    max_len = max(x.size(0) for x in xs)
    pad = xs[0].new(n_batch, max_len, * xs[0].size()[1:]).fill_(pad_value)
    for i in range(n_batch):
        pad[i, :xs[i].size(0)] = xs[i]
    return pad

def remove_pad(inputs, inputs_lengths):
    """
    Args:
        inputs: torch.Tensor, [B, C, T] or [B, T], B is batch size
        inputs_lengths: torch.Tensor, [B]
    Returns:
        results: a list containing B items, each item is [C, T], T varies
    """
    results = []
    dim = inputs.dim()
    if dim == 3:
        C = inputs.size(1)
    for input, length in zip(inputs, inputs_lengths):
        if dim == 3: # [B, C, T]
            results.append(input[:,:length].view(C, -1).cpu().numpy())
        elif dim == 2:  # [B, T]
            results.append(input[:length].view(-1).cpu().numpy())
    return results


def separate_process(xs):
    # perform padding and convert to tensor
    mixtures_pad = pad_list(xs, 0)
    
    mix_lengths = torch.tensor([len(item) for item in xs]) #torch.tensor([len(a) for a in xs])

#     print(mixtures_pad.numpy().tolist())
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps({'data': mixtures_pad.numpy().tolist()}).encode('utf-8')
    )
    response=json.loads(response['Body'].read())['response']
    response = torch.Tensor(response).cpu() 
    # Remove padding and flat
    flat_estimate = remove_pad(response, mix_lengths)
    return flat_estimate



In [72]:
# 由于语音分离耗时较长，采用分割并行处理的形式
filename = 'test.wav' #(10376000,)

instance_time = 1

x, fs = preprocess_wav(filename)
s_x = instance_time * fs

def batchify(input_np_arr, n = 1, frame_size = s_x): # 批量预测提速 默认n=1
    batch = []
    framed = librosa.util.frame(input_np_arr, frame_length=frame_size, hop_length=frame_size, axis=0)
    for i, fr in enumerate(framed):
        batch.append(torch.from_numpy(fr).float())
        if (i+1) % n == 0:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch
        
s1 = []
s2 = []
for batch in tqdm(batchify(x)):
    est = separate_process(batch)
    for b in range(len(est)):
        s1.extend([est[b][0]])
        s2.extend([est[b][1]])

s1 = np.concatenate(s1)
s2 = np.concatenate(s2)
s1 = s1 * 10
s2 = s2 * 10
write(s1, './complete_split_s1.wav')
write(s2, './complete_split_s2.wav')
print('saved')

10it [01:23,  8.37s/it]

saved


In [ ]:
# local train

# import sagemaker
# from sagemaker.local import LocalSession
# from sagemaker.pytorch import PyTorch
# import boto3
# from sagemaker import get_execution_role

# boto_session = boto3.Session(region_name='us-east-1')
# sagemaker_session = LocalSession(boto_session=boto_session)
# sagemaker_session.config = {'local': {'local_code': True}}
# role = get_execution_role()

# pytorch_estimator = PyTorch(entry_point='dprnn-train-deploy.py',
#                             instance_type='local',
#                             instance_count=1,
#                             image_uri='310850127430.dkr.ecr.us-east-1.amazonaws.com/asteroid:latest',
#                             sagemaker_session=sagemaker_session,
#                             role=role,
#                             source_dir='/home/ec2-user/SageMaker/asteroid_byoc/code')

# pytorch_estimator.fit('s3://dns4/cv')

In [ ]:
# local test

# import librosa
# from tqdm import tqdm
# from helper.utils import *
# from code.pred_utils import *
# import time
# from tqdm import tqdm as tqdm
# from asteroid.models import DPRNNTasNet

# def model_fn(model_dir):
#     model_path = os.path.join(model_dir, 'best_model.pth')
#     model = DPRNNTasNet.from_pretrained(model_path)

#     return model


# def predict_fn(input_data, model):
#     # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     # model.to(device)
#     model.eval()
#     model.cuda()
#     with torch.no_grad():
#         estimate_source = model(input_data)  # [B, C, T]

#     return estimate_source


# def preprocess_wav(filename, fs = 16000):
#     x, fs = librosa.load(filename, sr =fs)
#     x_mirror = x
#     asl_level = -26.0
#     y = x + x_mirror
# #     y = y/10**(asl_meter(y, fs)/20) * 10**(asl_level/20)
#     return y, fs


# def pad_list(xs, pad_value):
#     n_batch = len(xs)
#     max_len = max(x.size(0) for x in xs)
#     pad = xs[0].new(n_batch, max_len, * xs[0].size()[1:]).fill_(pad_value)
#     for i in range(n_batch):
#         pad[i, :xs[i].size(0)] = xs[i]
#     return pad

# def remove_pad(inputs, inputs_lengths):
#     """
#     Args:
#         inputs: torch.Tensor, [B, C, T] or [B, T], B is batch size
#         inputs_lengths: torch.Tensor, [B]
#     Returns:
#         results: a list containing B items, each item is [C, T], T varies
#     """
#     results = []
#     dim = inputs.dim()
#     if dim == 3:
#         C = inputs.size(1)
#     for input, length in zip(inputs, inputs_lengths):
#         if dim == 3: # [B, C, T]
#             results.append(input[:,:length].view(C, -1).cpu().numpy())
#         elif dim == 2:  # [B, T]
#             results.append(input[:length].view(-1).cpu().numpy())
#     return results

# def separate_process(xs):
#     # perform padding and convert to tensor
#     mixtures_pad = pad_list(xs, 0).cuda()
    
#     mix_lengths = torch.tensor([len(item) for item in xs]) #torch.tensor([len(a) for a in xs])
#     # Forward
#     response = predict_fn(mixtures_pad, model)
#     # Remove padding and flat
#     flat_estimate = remove_pad(response, mix_lengths)
#     return flat_estimate

# def batchify(input_np_arr, n = 8, frame_size = s_x):
#     batch = []
#     framed = librosa.util.frame(input_np_arr, frame_length=frame_size, hop_length=frame_size, axis=0)
#     for i, fr in enumerate(framed):
#         batch.append(torch.from_numpy(fr).float())
#         if (i+1) % n == 0:
#             yield batch
#             batch = []
#     if len(batch) > 0:
#         yield batch
        
# model = model_fn('./code/')

# filename = 'test.wav' 

# instance_time = 1

# x, fs = preprocess_wav(filename)
# s_x = instance_time * fs

# s1 = []
# s2 = []

# for batch in tqdm(batchify(x)):
#     est = separate_process(batch)
#     for b in range(len(est)):
#         s1.extend([est[b][0]])
#         s2.extend([est[b][1]])

# s1 = np.concatenate(s1)
# s2 = np.concatenate(s2)
# s1 = s1 * 10
# s2 = s2 * 10
# write(s1, './complete_split_s1.wav')
# write(s2, './complete_split_s2.wav')



In [ ]:

# est=np.load('test.npy')
# # response = client.invoke_endpoint(
# #     EndpointName=endpoint_name,
# #     ContentType="application/json",
# #     Body=json.dumps({'data': data.tolist()}).encode('utf-8')
# # )
# # local test
# from io import BytesIO
# import io
# import base64
# import json
# import requests
# dictionary = {'data':est.tolist()}
# jsonString = json.dumps(dictionary, indent=4)
# res = requests.post("http://localhost:8080/invocations", data = jsonString) 


In [58]:
# response = client.invoke_endpoint(
#     EndpointName='endpoint-asteroid2022-02-15-14-54-45-2022-02-15-14-54-45',
#     ContentType="application/json",
# #         Body=json.dumps({'data': mixtures_pad.numpy().tolist()}).encode('utf-8')
#     Body=json.dumps({'data': est.tolist()}).encode('utf-8')
# )

In [59]:
# test = response['Body'].read()

In [73]:
# type(json.loads(test)['response'])